In [143]:
import gzip, ujson, random
import toolz
import cPickle as c
import nltk.tokenize
import sklearn as sk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline

def getjsontext(d):
    return [toolz.keyfilter(lambda k: k == "text", d)]

def load_json_yelp(filepath):
    data=dict()
    data['text']=list()
    data['stars']=list()
    with gzip.open(filepath, 'rb') as f:
        for line in f:
            pick = random.randint(0,99) 
            if pick==0: #try to reduce the number of sample by randomly picking 5% of all data
                temp = ujson.loads(line)
                #data['text'].append(toolz.keyfilter(lambda k: k == "text", line))
                data['text'].append(temp['text'].encode('utf-8'))
                data['stars'].append(temp['stars'])
    return data

def tokenizereview(str):
    return nltk.tokenize.sent_tokenize(str)

#data loading
filepath = 'yelp_train_academic_dataset_review.json.gz'
yelp_data = load_json_yelp(filepath)

In [144]:
import dill
print len(yelp_data['text'])
print len(yelp_data['stars'])
#testvec = CountVectorizer()
#X_ct = testvec.fit_transform(yelp_data['text'])

10103
10103


In [153]:
#for problem one
#the place for class definition
class  Q1Estimator(sk.base.BaseEstimator, sk.base.RegressorMixin):
    def __init__(self,model):
        self.est = model
        pass
    def fit(self,X,y):
        test_param = {'alpha':[1,3,5,7,9,10]}
        gscv = GridSearchCV(self.est, test_param)
        gscv.fit(X,y)
        self.est = gscv.best_estimator_
        print gscv.best_params_ 
        #self.est.fit(X,y)
        return self
    
    def score(self,X,y):
        return self.est.score(X,y)
    
    def predict(self, X):
        return self.est.predict(X)
    

class Q1Transformer(sk.base.BaseEstimator, sk.base.TransformerMixin):
    def __init__(self):
        #self.vectorizer = CountVectorizer(max_df=0.99, min_df=0.01,stop_words=nltk.corpus.stopwords.words('english'))
        #self.vectorizer = CountVectorizer(stop_words=nltk.corpus.stopwords.words('english'))
        self.vectorizer = CountVectorizer()
        
    def fit(self,X,y=None):
        X=map(lambda x:x.lower(),X)
        self.vectorizer.fit_transform(X)
        return self
    
    def transform(self,X):
        X=map(lambda x:x.decode('utf-8').encode('utf-8').lower(),X)
        X_trans = self.vectorizer.transform(X)
        return X_trans

In [154]:
pipeline = Pipeline([('vectorize',Q1Transformer()),('estimate',Q1Estimator(linear_model.Ridge()))])
X=yelp_data['text']
y=yelp_data['stars']
presult = pipeline.fit(X,y)

with open('bow_model','wb') as output:
    dill.dump(presult,output)

{'alpha': 10}


In [155]:
with open('bow_model','rb') as instrm:
    datastruct = dill.load(instrm)
print datastruct

presult = datastruct.predict(X)

print datastruct.score(X,y)

Pipeline(steps=[('vectorize', Q1Transformer()), ('estimate', Q1Estimator(model=None))])
0.718139232354


In [126]:
#for problem two
#the place for class definition
from sklearn.feature_extraction.text import TfidfVectorizer
class  NormalEstimator(sk.base.BaseEstimator, sk.base.RegressorMixin):
    def __init__(self,model):
        self.est = model
        pass
    def fit(self,X,y):
        test_param = {'alpha':[0.01,0.05,0.1,0.3,0.5,1,10]}
        gscv = GridSearchCV(self.est, test_param)
        gscv.fit(X,y)
        self.est = gscv.best_estimator_
        print gscv.best_params_ 
        #self.est.fit(X,y)
        return self
    
    def score(self,X,y):
        return self.est.score(X,y)
    
    def predict(self, X):
        return self.est.predict(X)
    

class NormalTransformer(sk.base.BaseEstimator, sk.base.TransformerMixin):
    def __init__(self):
        #self.vectorizer = TfidfVectorizer(max_df=0.99, min_df=0.01,stop_words=nltk.corpus.stopwords.words('english'))
        self.vectorizer = TfidfVectorizer(stop_words=nltk.corpus.stopwords.words('english'))
        
    def fit(self,X,y=None):
        X = map(lambda x:x.lower(),X)
        self.vectorizer.fit_transform(X)
        return self
    
    def transform(self,X):
        if type(X) == dict:
            X['text'].lower()
            X_trans = self.vectorizer.transform(X['text'])
        else:
            X=map(lambda x:x.lower(),X)
            X_trans = self.vectorizer.transform(X)
        return X_trans

In [127]:
pipeline = Pipeline([('vectorize2',NormalTransformer()),('estimate2',NormalEstimator(linear_model.Ridge()))])
X=yelp_data['text']
y=yelp_data['stars']
presult = pipeline.fit(X,y)

with open('tfidf_model','wb') as output:
    dill.dump(presult,output)

{'alpha': 1}


In [128]:
with open('tfidf_model','rb') as instrm:
    datastruct2 = dill.load(instrm)
print datastruct2

presult = datastruct2.predict(X)

print datastruct2.score(X,y)

Pipeline(steps=[('vectorize2', NormalTransformer()), ('estimate2', NormalEstimator(model=None))])
0.760428898228
